In [4]:
import torch
from torch import nn
from torch.nn import functional as F
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256,10))

X = torch.rand(2,20)
print(X)
net(X)

tensor([[0.8876, 0.1334, 0.6111, 0.0057, 0.0192, 0.3799, 0.2161, 0.8459, 0.2224,
         0.3825, 0.3070, 0.2661, 0.7501, 0.2896, 0.1779, 0.8660, 0.3644, 0.3591,
         0.4851, 0.4285],
        [0.9819, 0.6834, 0.6290, 0.6682, 0.3998, 0.7408, 0.2923, 0.4500, 0.5798,
         0.4909, 0.3907, 0.6626, 0.7708, 0.2326, 0.2396, 0.5578, 0.2143, 0.1209,
         0.8254, 0.7266]])


tensor([[ 0.0924,  0.0684,  0.0050, -0.0062,  0.0203, -0.0071, -0.2480,  0.0500,
         -0.0856,  0.1289],
        [-0.0222, -0.0348, -0.0796, -0.0485,  0.1262, -0.0565, -0.3394,  0.1388,
         -0.1128,  0.2120]], grad_fn=<AddmmBackward>)

## A custom block

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [7]:
net = MLP()
net(X)

tensor([[ 0.0861, -0.1247, -0.2825, -0.1465,  0.0531, -0.1004, -0.1307, -0.1246,
         -0.0225, -0.1043],
        [ 0.1044, -0.0895, -0.2892, -0.1082,  0.0543,  0.0250, -0.1691, -0.1606,
          0.0698, -0.1879]], grad_fn=<AddmmBackward>)

## A sequential block

In [9]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # Here, `block` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[block] = block
            
    def forward(self, X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        for block in self._modules.values():
            X = block(X)
        return X

In [10]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0126,  0.0057,  0.1424, -0.0435, -0.1293,  0.0715, -0.0451,  0.1592,
          0.1003,  0.0553],
        [ 0.0749, -0.1080,  0.3019, -0.0042, -0.2257,  0.1350, -0.0580,  0.2217,
          0.1635, -0.0461]], grad_fn=<AddmmBackward>)